In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, AveragePooling2D, SeparableConv2D
from keras.layers import Activation, Flatten, Dense, BatchNormalization, Dropout
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras import optimizers
from keras.callbacks import LambdaCallback 
from keras import backend as K 
import tempfile 
import sys

### Create all the config parameters

In [0]:
img_width, img_height = 224, 224 

train_dir = "/content/drive/My Drive/Datasets/image_quality/training/"

valid_dir = "/content/drive/My Drive/Datasets/image_quality/evaluation/"

trained_model_path = "/content/drive/My Drive/Datasets/image_quality/model.h5"

training_plot_path = "/content/drive/My Drive/Datasets/image_quality/model_training_plot.png"

lr_loss_plot_path = "/content/drive/My Drive/Datasets/image_quality/lr_loss_plot.png"

num_classes = 2

num_train_samples = 1290

num_test_samples = 207 

epochs = 50

batch_size = 32 

input_shape = (img_width, img_height, 3)


### Create CNN model class

In [0]:
import os 
os.path.exists("/content/drive/My Drive/Datasets/image_quality/evaluation/")

True

In [0]:
# define a class which creates a CNN model 
class ConvNetModel:
  """Simple sequential Conv-net model"""
  def __init__(self, num_classes, input_shape):
      self.num_classes = num_classes 
      self.input_shape = input_shape

  @staticmethod
  def conv_block(x, num_filters, kernels=(3,3), strides=(1, 1), is_norm_conv=True):
      if is_norm_conv:
          x = Conv2D(num_filters, 
                     kernels,
                     strides=strides,
                     padding="same", 
                     kernel_initializer="he_normal", 
                     kernel_regularizer=l2(0.002))(x)
      else:
          x = SeparableConv2D(num_filters,
                              kernels,
                              strides=strides,
                              padding="same", 
                              depthwise_initializer="he_normal", 
                              pointwise_initializer="he_normal",
                              depthwise_regularizer=l2(0.002), 
                              pointwise_regularizer=l2(0.002))(x)

      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      return x

  def build(self):
      input_img = Input(shape = self.input_shape, name="input_image")
      x = ConvNetModel.conv_block(input_img, 32, kernels=(3,3), strides=(2, 2), is_norm_conv=False)
      x = Dropout(0.5)(x)
      x = ConvNetModel.conv_block(x, 64, kernels=(3,3), strides=(2, 2), is_norm_conv=False)
      x = Dropout(0.5)(x)
      x = ConvNetModel.conv_block(x, 128, kernels=(3,3), strides=(2, 2), is_norm_conv=False)
      x = Dropout(0.5)(x)
      x = ConvNetModel.conv_block(x, 256, kernels=(3,3), strides=(2, 2), is_norm_conv=False)
      x = Dropout(0.5)(x)
      x = ConvNetModel.conv_block(x, 512, kernels=(3,3), strides=(2, 2), is_norm_conv=False)
      x = Dropout(0.5)(x)
      x = AveragePooling2D(7, 7)(x)
      x = Flatten()(x)
      x = Dense(self.num_classes)(x)
      output = Activation("softmax")(x)
      model = Model(inputs=input_img, outputs=output)
      model.summary()

      return model       

### Create, build and compile ConvNet model

In [0]:
# create and compile a model     
cnn_model = ConvNetModel(num_classes, input_shape)
model = cnn_model.build()

# initialze an optimizer
#opt = optimizers.Adam(decay=1e-6, amsgrad=False)
opt = optimizers.SGD(momentum=0.9)

# compile it with an optimizer
model.compile(loss='categorical_crossentropy', 
              optimizer= opt, 
              metrics = ['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
separable_conv2d_11 (Separab (None, 112, 112, 32)      155       
_________________________________________________________________
batch_normalization_11 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
activation_13 (Activation)   (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 112, 112, 32)      0         
_________________________________________________________________
separable_conv2d_12 (Separab (None, 56, 56, 64)        2400      
_________________________________________________________________
batch_normalization_12 (Batc (None, 56, 56, 64)        256 

### **Create data (augmentation) generators**

In [0]:
# generate augmented training data 
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.15, fill_mode="reflect")

# this is the augmentation configuration we will use for testing: only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 1290 images belonging to 2 classes.
Found 217 images belonging to 2 classes.


### **Find Learning Rates (min and max bounds)**

In [0]:
class LearningRateFinder:
  """ goal of this class is to provide a plot where effects of using a range 
  of learning rates on the loss is displayed. Some LRs are too low and some are 
  too high, therefore, with the help of this plot, we can find an optimal range of 
  LRs (minimum and maximum bounds). 

  Starting and ending LRs choosen for the plot are too low (where network is unable 
  to learn) or too large (where loss is too high). Therefore, a good range of min and 
  max LR bounds should be somewhere inside of it and that's the aim of this class. 

  A careful analysis of the plot is required to find the right bounds. At the end, 
  entire network will be then trained by using correct learning rate (max learning rate) or min and max LRs. 
  """

  def __init__(self, model, stopFactor=4, beta=0.98):
      """ initializes variables for finding the learning rates 

          :param model: model for which learning rates and losses will be analyzed
          :param stopFactor: stop factor when the learning rate becomes too large to stop the model training process
          :param beta: used for averaging the loss value
          :param lrs: a list of tried LR values
          :param losses: a list of tried loss values
          :param avgLoss: average loss value over time 
          :param batchNum: current batch number 
          :param bestLoss: best low (of course lowest) found so far during training
          :param lrMult: learning rate multiplication factor 
          :weightsFile: filename to save initial (original) weights of the model
      """
      self.model = model
      self.stopFactor = stopFactor # not clear why 4? 

      self.beta = beta 
      self.lrs = [] # list of LRs which have been used already
      self.losses = [] # list of losses so far in the batch updates
      self.avgLoss = 0
      self.batchNum = 0 # current batch number/index

      self.bestLoss = 1e9 
      self.lrMult = 1
      self.weightsFile = None

  def reset(self):
      """ reset or re-initialize all variables from our constructor """
      self.lrs = []
      self.losses = []
      self.lrMult = 1
      self.avgLoss = 0 
      self.bestLoss = 1e9 
      self.batchNum = 0 
      self.weightsFile = None

  def is_data_iter(self, data):
      # define the set of class types we will check for
      iterClasses = ["NumpyArrayIterator", "DirectoryIterator", 
                     "DataFrameIterator", "Iterator", "Sequence"]
      # return whether our data is an iterator
      return data.__class__.__name__ in iterClasses

  def on_batch_end(self, batch, logs):
      """ responsible for setting/updating the new learning rate (LR) based on the current LR 
          and also, for recording current loss and LR. 
      """

      # get the current LR from the model and save to a list of LRs which have been used already 
      lr = K.get_value(self.model.optimizer.lr)
      self.lrs.append(lr)

      # update the batch number 
      self.batchNum += 1

      # get the loss at the end of this batch, compute the average loss, smooth it and save to a list 
      # of losses 
      l = logs["loss"]
      self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta)*l)
      smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
      self.losses.append(smooth)

      # compute maximum loss as a factor of best loss to stop the training 
      stopLoss = self.stopFactor * self.bestLoss 

      # check whether the loss has grown too large, therefore, stop the training 
      if self.batchNum > 1 and smooth > stopLoss: 
          self.model.stop_training = True 
          return 

      # check if the best loss should be updated 
      if self.batchNum == 1 or smooth < self.bestLoss:
          self.bestLoss = smooth 

      # finally increase the LR and set it as new LR for model training 
      lr *= self.lrMult 
      K.set_value(self.model.optimizer.lr, lr)

  def find(self, trainData, startLR, endLR, epochs=None, 
           stepsPerEpoch=None, batchSize=32, sampleSize=2048, 
           verbose=1):
      """ finds different learning rates (including optimal one) via model training 
          and at each batch update, new LR is update and saves current LR and loss. 

          :param trainData: training data either Numpy array data or data generator 
          :param startLR: starting learning rate 
          :param endLR: last learning rate 
          :param epochs: number of epochs to train for (if not value provided, it is calculate on a default sampleSize)
          :param stepsPerEpoch: total number of batch update steps per epoch 
          :sampleSize: size of training data to use when finding the optimal learning rate 
      """

      # reset class specific variables 
      self.reset()

      # determine if we are using a data generator or not 
      useGen = self.is_data_iter(trainData)

      # if we are using a generator and the steps_per_epoch is not supplied, 
      # raise an error 
      if useGen and stepsPerEpoch is None:
          raise Exception ("Using generator without supplying steps_per_epoch")

      # if we are not using a generator then our entire dataset must already be 
      # in memory 
      elif not useGen:
          # get number of samples in the training data and then derive the number 
          # of steps_per_epoch 
          numSamples = len(trainData[0])
          stepsPerEpoch = np.ceil(numSamples / float(batchSize))

      # if number of epochs is not provided, then compute it based on a 
      # default sample size 
      if epochs is None:
          epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

      # calculate total number of batch updates that will take place 
      numBatchUpdates = epochs * stepsPerEpoch 

      # derive the LR multiplier based on ending LR, starting LR and total 
      # number of batch updates (exponentially increase LR)
      self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

      # save the model's original weights, so we can reset the weights when we are 
      # done finiding the optimal learning rates 
      self.weightsFile = tempfile.mkstemp()[1] 
      self.model.save_weights(self.weightsFile)

      # save the model's original LR and then set the new "starting" learning rate 
      origLR = K.get_value(self.model.optimizer.lr)
      K.set_value(self.model.optimizer.lr, startLR)

      # perform model training and at each batch: update LR and record current LR and loss 
      # create a callback that will be called at the end of each batch, which increases the LR and 
      # save current LR and loss
      callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

      # check if we are using a data iterator 
      if useGen:
          self.model.fit_generator(
                  trainData,
                  steps_per_epoch=stepsPerEpoch,
                  epochs=epochs,
                  verbose=verbose,
                  callbacks=[callback])

      # otherwise, our entire training data is already in memory 
      else:
          self.model.fit(
                  trainData[0], trainData[1],
                  batch_size=batchSize,
                  epochs=epochs,
                  callbacks=[callback],
                  verbose=verbose)

      # finally, when we are done, set back the original model's weights and LR values 
      self.model.load_weights(self.weightsFile)
      K.set_value(self.model.optimizer.lr, origLR)

  def plot_loss(self, skipBegin=10, skipEnd=1, title="learning rate finder"):
      """ plot learning rates versus losses diagram """
      lrs = self.lrs[skipBegin:-skipEnd]
      losses = self.losses[skipBegin:-skipEnd]

      # plot the learning rate versus loss
      plt.plot(lrs, losses)
      plt.xscale("log")
      plt.xlabel("Learning Rate")
      plt.ylabel("Loss")
      plt.title(title)

In [0]:
lrf = LearningRateFinder(model)
lrf.find(
    train_generator, 
    startLR=1e-10, 
    endLR=1e+1, 
    epochs=10,
    stepsPerEpoch=num_train_samples // batch_size, 
    batchSize=batch_size)

# plot the loss for the various learning rates and save the
# resulting plot to disk
lrf.plot_loss()
plt.savefig(lr_loss_plot_path)

# gracefully exit the script so we can adjust our learning rates
# in the config and then train the network for our full set of
# epochs
print("[INFO] learning rate finder complete")
print("[INFO] examine plot and adjust learning rates before training")
sys.exit(0)

Epoch 1/10
40/40 [==============================] - 582s 15s/step - loss: 4.9070 - acc: 0.5103
Epoch 2/10
40/40 [==============================] - 145s 4s/step - loss: 4.8896 - acc: 0.5168
Epoch 3/10
40/40 [==============================] - 132s 3s/step - loss: 4.9008 - acc: 0.5164
Epoch 4/10
28/40 [====================>.........] - ETA: 39s - loss: 4.8944 - acc: 0.5121

### **Define a learning rate scheduler**

In [0]:
class PolynomialDecay():
	def __init__(self, maxEpochs=50, initAlpha=0.01, power=1.0):
		# store the maximum number of epochs, base learning rate,
		# and power of the polynomial
		self.maxEpochs = maxEpochs
		self.initAlpha = initAlpha
		self.power = power
 
	def __call__(self, epoch):
		# compute the new learning rate based on polynomial decay
		decay = (1 - (epoch / float(self.maxEpochs))) ** self.power
		alpha = self.initAlpha * decay
 
		# return the new learning rate
		return float(alpha)

### **Define all the necessary callbacks and train the model**

In [0]:
# define callbacks before starting the training
early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=0.00001, verbose=1)
model_checkpoint = ModelCheckpoint(trained_model_path, monitor="val_acc", save_best_only=True, mode='max', verbose=1)
#schedule = PolynomialDecay(maxEpochs=epochs, initAlpha=1e-1, power=5)

#callbacks = [reduce_lr, early_stop, model_checkpoint, LearningRateScheduler(schedule)]
callbacks = [reduce_lr, early_stop, model_checkpoint]

# perform model training 
H = model.fit_generator(generator=train_generator, 
                    steps_per_epoch=num_train_samples // batch_size, 
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=num_test_samples // batch_size, 
                    callbacks=callbacks)

Epoch 1/50
40/40 [==============================] - 126s 3s/step - loss: 4.1560 - acc: 0.5698 - val_loss: 3.6153 - val_acc: 0.5469

Epoch 00001: val_acc improved from -inf to 0.54688, saving model to /content/drive/My Drive/Datasets/image_quality/model.h5
Epoch 2/50
40/40 [==============================] - 126s 3s/step - loss: 3.2462 - acc: 0.5775 - val_loss: 2.9006 - val_acc: 0.5568

Epoch 00002: val_acc improved from 0.54688 to 0.55676, saving model to /content/drive/My Drive/Datasets/image_quality/model.h5
Epoch 3/50
40/40 [==============================] - 126s 3s/step - loss: 2.6472 - acc: 0.5923 - val_loss: 2.5706 - val_acc: 0.5568

Epoch 00003: val_acc improved from 0.55676 to 0.55676, saving model to /content/drive/My Drive/Datasets/image_quality/model.h5
Epoch 4/50
40/40 [==============================] - 122s 3s/step - loss: 2.2435 - acc: 0.6079 - val_loss: 2.0982 - val_acc: 0.5838

Epoch 00004: val_acc improved from 0.55676 to 0.58378, saving model to /content/drive/My Drive

KeyboardInterrupt: ignored

## **Plot the results**

In [0]:
# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()
plt.savefig(training_plot_path)

NameError: ignored

<Figure size 432x288 with 0 Axes>